In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import os
import glob
import openpyxl

In [2]:
os.getcwd()

'/home/grace/Documents/python/tsca_categories/notebooks'

In [3]:

TOP = os.getcwd().replace('notebooks', '')
TOP

'/home/grace/Documents/python/tsca_categories/'

In [4]:

raw_dir = TOP + 'data/raw/'
processed_dir = TOP +'data/processed/'
interim_dir = TOP + 'data/interim/'
external_dir = TOP + 'data/external/'
figures_dir = TOP + 'reports/figures/'

In [5]:
df = pd.read_excel(raw_dir+'TSCA_list_chemicals-2022-07-07-13-54-23.xlsx')

In [6]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole, MolsToGridImage

#Show mols in dataframes
from rdkit.Chem import PandasTools
from rdkit import Chem
from rdkit.Chem.Draw import MolsToGridImage
from IPython.core.display import HTML
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import PandasTools
from rdkit.Chem import AllChem
from rdkit.Chem import rdDepictor
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit import DataStructs

In [7]:
df.shape

(14247, 14)

In [8]:
df.rename(columns = {'DTXSID' : 'dtxsid'}, inplace = True)

#### Merge physchem predictions from OPERA Version 2.8.4

In [9]:
opera_df = pd.read_csv(processed_dir+'TSCA_JUL22_v3-sdf_OPERA2.8Pred.csv')

In [10]:
opera_df.rename(columns = {'MoleculeID' : 'dtxsid'}, inplace = True)

In [11]:
opera_df.columns

Index(['dtxsid', 'MolWeight', 'nbAtoms', 'nbHeavyAtoms', 'nbC', 'nbO', 'nbN',
       'nbAromAtom', 'nbRing', 'nbHeteroRing', 'Sp3Sp2HybRatio', 'nbRotBd',
       'nbHBdAcc', 'ndHBdDon', 'nbLipinskiFailures', 'TopoPolSurfAir',
       'MolarRefract', 'CombDipolPolariz', 'LogP_pred', 'LogP_predRange',
       'AD_LogP', 'AD_index_LogP', 'Conf_index_LogP', 'MP_pred',
       'MP_predRange', 'AD_MP', 'AD_index_MP', 'Conf_index_MP', 'BP_pred',
       'BP_predRange', 'AD_BP', 'AD_index_BP', 'Conf_index_BP', 'LogVP_pred',
       'VP_predRange', 'AD_VP', 'AD_index_VP', 'Conf_index_VP', 'LogWS_pred',
       'WS_predRange', 'AD_WS', 'AD_index_WS', 'Conf_index_WS', 'LogHL_pred',
       'HL_predRange', 'AD_HL', 'AD_index_HL', 'Conf_index_HL', 'RT_pred',
       'AD_RT', 'AD_index_RT', 'Conf_index_RT', 'RT_predRange', 'LogKOA_pred',
       'KOA_predRange', 'AD_KOA', 'AD_index_KOA', 'Conf_index_KOA',
       'ionization', 'pKa_a_pred', 'pKa_a_predRange', 'pKa_b_pred',
       'pKa_b_predRange', 'AD_pKa', '

In [12]:
df2 = pd.merge(df, opera_df, on = 'dtxsid', how = 'left')

In [13]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14247 entries, 0 to 14246
Data columns (total 86 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   dtxsid                     14247 non-null  object 
 1   PREFERRED_NAME             14247 non-null  object 
 2   CASRN                      14247 non-null  object 
 3   INCHIKEY                   14247 non-null  object 
 4   IUPAC_NAME                 14247 non-null  object 
 5   SMILES                     14247 non-null  object 
 6   INCHI_STRING               14247 non-null  object 
 7   MOLECULAR_FORMULA          14247 non-null  object 
 8   AVERAGE_MASS               14247 non-null  float64
 9   MONOISOTOPIC_MASS          14247 non-null  float64
 10  DATA_SOURCES               14247 non-null  int64  
 11  NUMBER_OF_PUBMED_ARTICLES  14247 non-null  object 
 12  PUBCHEM_DATA_SOURCES       14247 non-null  object 
 13  CPDAT_COUNT                14247 non-null  obj

#### Tag those substances for which predictions were not performed for various reasons or where RDKit could not transform the SMILES to SDF format or there were no SMILES to start with.

In [14]:
discard_df = pd.read_csv(processed_dir+'TSCA_JUL22_v3_DiscardedStructures.csv')

In [15]:
discard_dict = {k:v for k,v in zip(discard_df['Molecule name'], discard_df['Error'])}
#discard_dict

In [16]:
rdkit_errors = pd.read_csv(processed_dir+'rdkit_errors.csv')

In [17]:
rdkit_list = rdkit_errors['dtxsid'].tolist()

In [18]:
df2

,dtxsid,PREFERRED_NAME,CASRN,INCHIKEY,IUPAC_NAME,SMILES,INCHI_STRING,MOLECULAR_FORMULA,AVERAGE_MASS,MONOISOTOPIC_MASS,...,AD_pKa,AD_index_pKa,Conf_index_pKa,LogD55_pred,LogD55_predRange,LogD74_pred,LogD74_predRange,AD_LogD,AD_index_LogD,Conf_index_LogD
0,DTXSID3060164,Benzhydrylbenzene,519-73-3,AAAQKTZKLRYKHR-UHFFFAOYSA-N,"1,1',1''-Methanetriyltribenzene",C1=CC=C(C=C1)C(C1=CC=CC=C1)C1=CC=CC=C1,InChI=1S/C19H16/c1-4-10-16(11-5-1)19(17-12-6-2...,C19H16,244.337,244.125201,...,0.0,0.000,0.000,5.76,[5.56:5.96],5.76,[5.56:5.96],0.0,0.3140,0.3825
1,DTXSID7060837,"Propane, 1,3-diiodo-",627-31-6,AAAXMNYUNVCMCJ-UHFFFAOYSA-N,"1,3-Diiodopropane",ICCCI,InChI=1S/C3H6I2/c4-2-1-3-5/h1-3H2,C3H6I2,295.890,295.855890,...,0.0,0.000,0.000,3.02,[2.99:3.05],3.02,[2.99:3.05],0.0,0.4990,0.4725
2,DTXSID9025879,"3,3'-(p-Phenylene)diacrylic acid",16323-43-6,AAFXQFIGKBLKMC-UHFFFAOYSA-N,"3,3'-(1,4-Phenylene)di(prop-2-enoic acid)",OC(=O)C=CC1=CC=C(C=CC(O)=O)C=C1,InChI=1S/C12H10O4/c13-11(14)7-5-9-1-2-10(4-3-9...,C12H10O4,218.208,218.057909,...,1.0,0.447,0.772,-0.14,[-4.93:4.65],-2.04,[-8.73:4.65],1.0,0.5125,0.7140
3,DTXSID2026282,C.I. Vat brown 3,131-92-0,AAKMSGQPNUGLAZ-UHFFFAOYSA-N,"N,N'-(5,10,15,17-Tetraoxo-10,15,16,17-tetrahyd...",O=C(NC1=CC=CC2=C1C(=O)C1=C(C3=C(C=C1)C1=C(N3)C...,InChI=1S/C42H23N3O6/c46-37-24-14-7-8-15-25(24)...,C42H23N3O6,665.661,665.158685,...,0.0,0.223,0.591,3.10,[0.56:5.64],2.70,[-1.74:7.14],0.0,0.2025,0.4450
4,DTXSID4052188,"2,5,5-Trimethyl-2-phenyl-1,3-dioxane",5406-58-6,AALXTPRRKXUUOM-UHFFFAOYSA-N,"2,5,5-Trimethyl-2-phenyl-1,3-dioxane",CC1(C)COC(C)(OC1)C1=CC=CC=C1,"InChI=1S/C13H18O2/c1-12(2)9-14-13(3,15-10-12)1...",C13H18O2,206.285,206.130680,...,0.0,0.000,0.000,2.72,[2.23:3.21],2.72,[2.23:3.21],0.0,0.2825,0.3070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14242,DTXSID5064209,2-(Carboxymethylthio)benzothiazole,6295-57-4,ZZUQWNYNSKJLPI-UHFFFAOYSA-N,"[(1,3-Benzothiazol-2-yl)sulfanyl]acetic acid",OC(=O)CSC1=NC2=CC=CC=C2S1,InChI=1S/C9H7NO2S2/c11-8(12)5-13-9-10-6-3-1-2-...,C9H7NO2S2,225.280,224.991821,...,1.0,0.504,0.778,-1.26,[-6.18:3.66],-3.16,[-9.98:3.66],1.0,0.5305,0.7580
14243,DTXSID7062873,"Butanoic acid, 4-chloro-, methyl ester",3153-37-5,ZZUYIRISBMWFMV-UHFFFAOYSA-N,Methyl 4-chlorobutanoate,COC(=O)CCCCl,"InChI=1S/C5H9ClO2/c1-8-5(7)3-2-4-6/h2-4H2,1H3",C5H9ClO2,136.580,136.029107,...,0.0,0.000,0.000,1.55,[1.13:1.96],1.55,[1.13:1.96],0.0,0.3150,0.3670
14244,DTXSID9040342,"4-Ethyl-1,3-dioxolan-2-one",4437-85-8,ZZXUZKXVROWEIF-UHFFFAOYSA-N,"4-Ethyl-1,3-dioxolan-2-one",CCC1COC(=O)O1,"InChI=1/C5H8O3/c1-2-4-3-7-5(6)8-4/h4H,2-3H2,1H3",C5H8O3,116.116,116.047344,...,0.0,0.000,0.000,0.03,[-0.64:0.72],0.03,[-0.64:0.72],0.0,0.2740,0.2945
14245,DTXSID3044889,"2,5-Di-tert-butylcyclohexa-2,5-diene-1,4-dione",2460-77-7,ZZYASVWWDLJXIM-UHFFFAOYSA-N,"2,5-Di-tert-butylcyclohexa-2,5-diene-1,4-dione",CC(C)(C)C1=CC(=O)C(=CC1=O)C(C)(C)C,"InChI=1S/C14H20O2/c1-13(2,3)9-7-12(16)10(8-11(...",C14H20O2,220.312,220.146330,...,0.0,0.000,0.000,4.26,[4.02:4.42],4.26,[4.02:4.42],0.0,0.3690,0.3790


In [19]:
mylst = []
for x in df2['dtxsid']:
    if x in discard_dict.keys():
        mylst.append([x, discard_dict[x]])
    if x in rdkit_list:
        mylst.append([x, 'rdkit_error'])
    

In [20]:
errors = {k:v for k,v in zip(pd.DataFrame(mylst)[0], pd.DataFrame(mylst)[1])}

In [21]:
df2['errors'] = df2['dtxsid'].replace(errors)

In [22]:
df2['errors'] = df2['errors'].apply(lambda x: np.nan if "DTX" in x else x)

In [23]:
df2[df2['errors'].notnull()]

,dtxsid,PREFERRED_NAME,CASRN,INCHIKEY,IUPAC_NAME,SMILES,INCHI_STRING,MOLECULAR_FORMULA,AVERAGE_MASS,MONOISOTOPIC_MASS,...,AD_index_pKa,Conf_index_pKa,LogD55_pred,LogD55_predRange,LogD74_pred,LogD74_predRange,AD_LogD,AD_index_LogD,Conf_index_LogD,errors
15,DTXSID8065612,Thulium fluoride (TmF3),13760-79-7,AATUHDXSJTXIHB-UHFFFAOYSA-K,Thulium trifluoride,F[Tm](F)F,InChI=1S/3FH.Tm/h3*1H;/q;;;+3/p-3,F3Tm,225.92943,225.929430,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Inorganic
30,DTXSID2049715,Phosphonic acid,13598-36-2,ABLZXFCXXLZCGV-UHFFFAOYSA-N,Phosphonic acid,OP(O)=O,"InChI=1S/H3O3P/c1-4(2)3/h4H,(H2,1,2,3)",H3O3P,81.99500,81.981981,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Inorganic
40,DTXSID0064830,"Silane, tetrafluoro-",7783-61-1,ABTOQLMXBSRXSM-UHFFFAOYSA-N,Tetrafluorosilane,F[Si](F)(F)F,"InChI=1S/F4Si/c1-5(2,3)4",F4Si,104.07900,103.970539,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Inorganic
55,DTXSID4028042,Methyltin tris(2-ethylhexyl mercaptoacetate),57583-34-3,ACRNSUGLVQJCOM-UHFFFAOYSA-K,2-Ethylhexyl 10-ethyl-4-({2-[(2-ethylhexyl)oxy...,CCCCC(CC)COC(=O)CS[Sn](C)(SCC(=O)OCC(CC)CCCC)S...,InChI=1/3C10H20O2S.CH3.Sn/c3*1-3-5-6-9(4-2)7-1...,C31H60O6S3Sn,743.71000,744.257405,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Organometallics
60,DTXSID90873902,Phosphinic acid,6303-21-5,ACVYVLVWPXVTIT-UHFFFAOYSA-N,Phosphinic acid,[H]P([H])(O)=O,"InChI=1S/H3O2P/c1-3-2/h3H2,(H,1,2)",H3O2P,65.99600,65.987066,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Inorganic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14175,DTXSID40883839,"Magnesium, chloroethyl-",2386-64-3,ZWNJJACKPNTXHY-UHFFFAOYSA-M,Chlorido(ethyl)magnesium,CC[Mg]Cl,"InChI=1S/C2H5.ClH.Mg/c1-2;;/h1H2,2H3;1H;/q;;+1...",C2H5ClMg,88.82000,87.993020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Organometallics
14189,DTXSID3064930,Titanium chloride (TiCl2),10049-06-6,ZWYDDDAMNQQZHD-UHFFFAOYSA-L,Titanium(2+) dichloride,Cl[Ti]Cl,InChI=1S/2ClH.Ti/h2*1H;/q;;+2/p-2,Cl2Ti,118.77000,117.885647,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Inorganic
14193,DTXSID1061441,"Stannane, dibutyldimethoxy-",1067-55-6,ZXDVQYBUEVYUCG-UHFFFAOYSA-N,Dibutyl(dimethoxy)stannane,CCCC[Sn](CCCC)(OC)OC,InChI=1S/2C4H9.2CH3O.Sn/c2*1-3-4-2;2*1-2;/h2*1...,C10H24O2Sn,295.01000,296.079832,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Organometallics
14212,DTXSID2037136,Uranium nitrate oxide (UO2(NO3)2),10102-06-4,ZYEWBKAZYICUMJ-UHFFFAOYSA-N,Bis(nitrato-kappaO)[bis(oxido)]uranium,[O-][N+](=O)O[U](=O)(=O)O[N+]([O-])=O,InChI=1S/2NO3.2O.U/c2*2-1(3)4;;;/q2*-1;;;+2,N2O8U,394.03500,394.016250,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Inorganic


In [24]:
df2['errors'].value_counts()

 Inorganic          588
 Organometallics    289
rdkit_error          51
 Size                12
 Mixture              6
Name: errors, dtype: int64

In [25]:
df2[df2['SMILES'] == '-']

,dtxsid,PREFERRED_NAME,CASRN,INCHIKEY,IUPAC_NAME,SMILES,INCHI_STRING,MOLECULAR_FORMULA,AVERAGE_MASS,MONOISOTOPIC_MASS,...,AD_index_pKa,Conf_index_pKa,LogD55_pred,LogD55_predRange,LogD74_pred,LogD74_predRange,AD_LogD,AD_index_LogD,Conf_index_LogD,errors
5299,DTXSID80894904,Dilithium molybdate,13568-40-6,KCHZBGIOQNFCJY-UHFFFAOYSA-N,-,-,InChI=1S/2Li.Mo.2HO.2O/h;;;2*1H;;,H2Li2MoO4,175.840,177.932720,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7824,DTXSID70924244,Arsanylidynegermyl,12271-72-6,OEOKQRBZALRLDT-UHFFFAOYSA-N,Arsanylidynegermyl,-,InChI=1S/AsGe/c1-2,AsGe,147.552,148.842773,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
for index in df2.index:
    if df2.loc[index, 'SMILES' ] == '-':
        df2.loc[index, 'errors'] = 'no_smiles'

In [27]:
df2['errors'].value_counts()

 Inorganic          588
 Organometallics    289
rdkit_error          51
 Size                12
 Mixture              6
no_smiles             2
Name: errors, dtype: int64

In [28]:
df2.columns

Index(['dtxsid', 'PREFERRED_NAME', 'CASRN', 'INCHIKEY', 'IUPAC_NAME', 'SMILES',
       'INCHI_STRING', 'MOLECULAR_FORMULA', 'AVERAGE_MASS',
       'MONOISOTOPIC_MASS', 'DATA_SOURCES', 'NUMBER_OF_PUBMED_ARTICLES',
       'PUBCHEM_DATA_SOURCES', 'CPDAT_COUNT', 'MolWeight', 'nbAtoms',
       'nbHeavyAtoms', 'nbC', 'nbO', 'nbN', 'nbAromAtom', 'nbRing',
       'nbHeteroRing', 'Sp3Sp2HybRatio', 'nbRotBd', 'nbHBdAcc', 'ndHBdDon',
       'nbLipinskiFailures', 'TopoPolSurfAir', 'MolarRefract',
       'CombDipolPolariz', 'LogP_pred', 'LogP_predRange', 'AD_LogP',
       'AD_index_LogP', 'Conf_index_LogP', 'MP_pred', 'MP_predRange', 'AD_MP',
       'AD_index_MP', 'Conf_index_MP', 'BP_pred', 'BP_predRange', 'AD_BP',
       'AD_index_BP', 'Conf_index_BP', 'LogVP_pred', 'VP_predRange', 'AD_VP',
       'AD_index_VP', 'Conf_index_VP', 'LogWS_pred', 'WS_predRange', 'AD_WS',
       'AD_index_WS', 'Conf_index_WS', 'LogHL_pred', 'HL_predRange', 'AD_HL',
       'AD_index_HL', 'Conf_index_HL', 'RT_pred', 'AD

#### Merge in QSAR READY SMILES from separate OPERA output file

In [29]:
qsar_read_smi = pd.read_csv(processed_dir+'TSCA_JUL22_v3_Summary_file.csv')

In [30]:
qsar_read = {k:v for k,v in zip(qsar_read_smi['Molecule name'], qsar_read_smi['Canonical_QSARr'])}

In [31]:
df2['qsar_ready_smiles'] = df2['dtxsid'].replace(qsar_read)

In [32]:
df2['qsar_ready_smiles'] = df2['qsar_ready_smiles'].apply(lambda x: np.nan if "DTX" in x else x)

In [33]:
#df2[(df2['errors'].isnull()) & (df2['qsar_ready_smiles'].notnull())]

In [34]:
df3 = df2.copy()

In [35]:
df3.columns

Index(['dtxsid', 'PREFERRED_NAME', 'CASRN', 'INCHIKEY', 'IUPAC_NAME', 'SMILES',
       'INCHI_STRING', 'MOLECULAR_FORMULA', 'AVERAGE_MASS',
       'MONOISOTOPIC_MASS', 'DATA_SOURCES', 'NUMBER_OF_PUBMED_ARTICLES',
       'PUBCHEM_DATA_SOURCES', 'CPDAT_COUNT', 'MolWeight', 'nbAtoms',
       'nbHeavyAtoms', 'nbC', 'nbO', 'nbN', 'nbAromAtom', 'nbRing',
       'nbHeteroRing', 'Sp3Sp2HybRatio', 'nbRotBd', 'nbHBdAcc', 'ndHBdDon',
       'nbLipinskiFailures', 'TopoPolSurfAir', 'MolarRefract',
       'CombDipolPolariz', 'LogP_pred', 'LogP_predRange', 'AD_LogP',
       'AD_index_LogP', 'Conf_index_LogP', 'MP_pred', 'MP_predRange', 'AD_MP',
       'AD_index_MP', 'Conf_index_MP', 'BP_pred', 'BP_predRange', 'AD_BP',
       'AD_index_BP', 'Conf_index_BP', 'LogVP_pred', 'VP_predRange', 'AD_VP',
       'AD_index_VP', 'Conf_index_VP', 'LogWS_pred', 'WS_predRange', 'AD_WS',
       'AD_index_WS', 'Conf_index_WS', 'LogHL_pred', 'HL_predRange', 'AD_HL',
       'AD_index_HL', 'Conf_index_HL', 'RT_pred', 'AD

#### Convert physchem properties HLC and VP and Water Solubility to relevant units to apply the testing track rules

In [36]:
df3['HLC'] = df3['LogHL_pred'].apply(lambda x: 10**x)

In [37]:
df3['WS_pred_mol/L'] = df3['LogWS_pred'].apply(lambda x: 10**x)

In [38]:
df3['WS_pred_mg/L'] = df3[['WS_pred_mol/L', 'MolWeight']].apply(lambda x: 1000*(x[0]*x[1]), axis = 1)

In [39]:
df3[df3['errors'].isnull()]

,dtxsid,PREFERRED_NAME,CASRN,INCHIKEY,IUPAC_NAME,SMILES,INCHI_STRING,MOLECULAR_FORMULA,AVERAGE_MASS,MONOISOTOPIC_MASS,...,LogD74_pred,LogD74_predRange,AD_LogD,AD_index_LogD,Conf_index_LogD,errors,qsar_ready_smiles,HLC,WS_pred_mol/L,WS_pred_mg/L
0,DTXSID3060164,Benzhydrylbenzene,519-73-3,AAAQKTZKLRYKHR-UHFFFAOYSA-N,"1,1',1''-Methanetriyltribenzene",C1=CC=C(C=C1)C(C1=CC=CC=C1)C1=CC=CC=C1,InChI=1S/C19H16/c1-4-10-16(11-5-1)19(17-12-6-2...,C19H16,244.337,244.125201,...,5.76,[5.56:5.96],0.0,0.3140,0.3825,NaN,C1=CC=CC=C1C(C1C=CC=CC=1)C1C=CC=CC=1,2.630268e-06,4.073803e-07,0.099452
1,DTXSID7060837,"Propane, 1,3-diiodo-",627-31-6,AAAXMNYUNVCMCJ-UHFFFAOYSA-N,"1,3-Diiodopropane",ICCCI,InChI=1S/C3H6I2/c4-2-1-3-5/h1-3H2,C3H6I2,295.890,295.855890,...,3.02,[2.99:3.05],0.0,0.4990,0.4725,NaN,ICCCI,2.454709e-03,7.413102e-04,219.321006
2,DTXSID9025879,"3,3'-(p-Phenylene)diacrylic acid",16323-43-6,AAFXQFIGKBLKMC-UHFFFAOYSA-N,"3,3'-(1,4-Phenylene)di(prop-2-enoic acid)",OC(=O)C=CC1=CC=C(C=CC(O)=O)C=C1,InChI=1S/C12H10O4/c13-11(14)7-5-9-1-2-10(4-3-9...,C12H10O4,218.208,218.057909,...,-2.04,[-8.73:4.65],1.0,0.5125,0.7140,NaN,OC(=O)C=CC1C=CC(C=CC(O)=O)=CC=1,6.025596e-10,9.120108e-03,1988.711764
3,DTXSID2026282,C.I. Vat brown 3,131-92-0,AAKMSGQPNUGLAZ-UHFFFAOYSA-N,"N,N'-(5,10,15,17-Tetraoxo-10,15,16,17-tetrahyd...",O=C(NC1=CC=CC2=C1C(=O)C1=C(C3=C(C=C1)C1=C(N3)C...,InChI=1S/C42H23N3O6/c46-37-24-14-7-8-15-25(24)...,C42H23N3O6,665.661,665.158685,...,2.70,[-1.74:7.14],0.0,0.2025,0.4450,NaN,O=C(NC1=CC2=C(NC3C4=C(C=CC2=3)C(=O)C2C(=CC=CC=...,2.630268e-09,2.454709e-08,0.016377
4,DTXSID4052188,"2,5,5-Trimethyl-2-phenyl-1,3-dioxane",5406-58-6,AALXTPRRKXUUOM-UHFFFAOYSA-N,"2,5,5-Trimethyl-2-phenyl-1,3-dioxane",CC1(C)COC(C)(OC1)C1=CC=CC=C1,"InChI=1S/C13H18O2/c1-12(2)9-14-13(3,15-10-12)1...",C13H18O2,206.285,206.130680,...,2.72,[2.23:3.21],0.0,0.2825,0.3070,NaN,CC1(C)COC(C)(OC1)C1C=CC=CC=1,9.772372e-05,1.148154e-03,236.669687
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14242,DTXSID5064209,2-(Carboxymethylthio)benzothiazole,6295-57-4,ZZUQWNYNSKJLPI-UHFFFAOYSA-N,"[(1,3-Benzothiazol-2-yl)sulfanyl]acetic acid",OC(=O)CSC1=NC2=CC=CC=C2S1,InChI=1S/C9H7NO2S2/c11-8(12)5-13-9-10-6-3-1-2-...,C9H7NO2S2,225.280,224.991821,...,-3.16,[-9.98:3.66],1.0,0.5305,0.7580,NaN,OC(=O)CSC1=NC2=CC=CC=C2S1,3.388442e-09,9.772372e-04,219.870381
14243,DTXSID7062873,"Butanoic acid, 4-chloro-, methyl ester",3153-37-5,ZZUYIRISBMWFMV-UHFFFAOYSA-N,Methyl 4-chlorobutanoate,COC(=O)CCCCl,"InChI=1S/C5H9ClO2/c1-8-5(7)3-2-4-6/h2-4H2,1H3",C5H9ClO2,136.580,136.029107,...,1.55,[1.13:1.96],0.0,0.3150,0.3670,NaN,COC(=O)CCCCl,2.818383e-04,2.041738e-01,27773.578977
14244,DTXSID9040342,"4-Ethyl-1,3-dioxolan-2-one",4437-85-8,ZZXUZKXVROWEIF-UHFFFAOYSA-N,"4-Ethyl-1,3-dioxolan-2-one",CCC1COC(=O)O1,"InChI=1/C5H8O3/c1-2-4-3-7-5(6)8-4/h4H,2-3H2,1H3",C5H8O3,116.116,116.047344,...,0.03,[-0.64:0.72],0.0,0.2740,0.2945,NaN,CCC1COC(=O)O1,2.570396e-08,9.772372e-01,113405.784063
14245,DTXSID3044889,"2,5-Di-tert-butylcyclohexa-2,5-diene-1,4-dione",2460-77-7,ZZYASVWWDLJXIM-UHFFFAOYSA-N,"2,5-Di-tert-butylcyclohexa-2,5-diene-1,4-dione",CC(C)(C)C1=CC(=O)C(=CC1=O)C(C)(C)C,"InChI=1S/C14H20O2/c1-13(2,3)9-7-12(16)10(8-11(...",C14H20O2,220.312,220.146330,...,4.26,[4.02:4.42],0.0,0.3690,0.3790,NaN,CC(C)(C)C1=CC(=O)C(=CC1=O)C(C)(C)C,5.370318e-04,3.981072e-03,876.418325


In [40]:
df4 = df3.set_index('dtxsid')

In [41]:
test_track = []
for i, row in df4.iterrows():
    if (row['MP_pred'] >25) & (row['WS_pred_mg/L'] < 0.001):
        test_track.append([i,'A'])
    elif (row['MP_pred'] >25) & (row['WS_pred_mg/L'] > -0.001):
        test_track.append([i,'B'])
    elif (row['BP_pred'] <25) & (row['WS_pred_mg/L'] < 0.001):
        test_track.append([i,'D'])
    elif (row['BP_pred'] <25) & (row['WS_pred_mg/L'] >=0.001) & (row['HLC'] <0.1):
        test_track.append([i,'C'])
    elif (row['BP_pred'] <25) & (row['WS_pred_mg/L'] >=0.001) & (row['HLC'] >=0.1):
        test_track.append([i,'D'])
    elif (row['MP_pred'] <=25) & (row['BP_pred'] >=25) & (row['WS_pred_mg/L'] >=0.001) & (10**row['LogVP_pred'] <=75):
        test_track.append([i,'B'])
    elif (row['MP_pred'] <=25) & (row['BP_pred'] >=25) & (row['WS_pred_mg/L'] <0.001):
        test_track.append([i, 'C'])
    elif (row['MP_pred'] <=25) & (row['BP_pred'] >=25) & (row['WS_pred_mg/L'] >0.001) & (10**row['LogVP_pred'] >75):
        test_track.append([i, 'C'])
                      

In [42]:
test_track = {k:v for k, v in zip(pd.DataFrame(test_track)[0], pd.DataFrame(test_track)[1])}

In [43]:
df3['test_track'] = df3['dtxsid'].replace(test_track)

In [44]:
df3.test_track.value_counts()

B                  12757
C                    449
D                     33
A                     14
DTXSID5037234          1
                   ...  
DTXSID201014482        1
DTXSID50893904         1
DTXSID20894221         1
DTXSID90894899         1
DTXSID0065981          1
Name: test_track, Length: 998, dtype: int64

In [45]:
df3['test_track'] = df3['test_track'].apply(lambda x: np.nan if "DTX" in x else x)

In [46]:
df3.test_track.value_counts(dropna = False)

B      12757
NaN      994
C        449
D         33
A         14
Name: test_track, dtype: int64

#### Can we provide a flag for substances to determine their physical form from the IAD guidance using Mpt and Bpt indicators https://www.epa.gov/sites/default/files/2015-05/documents/05-iad_discretes_june2013.pdf

In [47]:
def func(x,y):
    if x < 25 and y > 25:
        return "liquid"
    if x > 25:
        return "solid"
    if y < 25:
        return "gas"
    else:
        return np.nan

In [48]:
df3['physical_form'] = df3.apply(lambda x: func(x.MP_pred, x.BP_pred), axis =1)

In [49]:
df3[df3['errors'].notnull()]

,dtxsid,PREFERRED_NAME,CASRN,INCHIKEY,IUPAC_NAME,SMILES,INCHI_STRING,MOLECULAR_FORMULA,AVERAGE_MASS,MONOISOTOPIC_MASS,...,AD_LogD,AD_index_LogD,Conf_index_LogD,errors,qsar_ready_smiles,HLC,WS_pred_mol/L,WS_pred_mg/L,test_track,physical_form
15,DTXSID8065612,Thulium fluoride (TmF3),13760-79-7,AATUHDXSJTXIHB-UHFFFAOYSA-K,Thulium trifluoride,F[Tm](F)F,InChI=1S/3FH.Tm/h3*1H;/q;;;+3/p-3,F3Tm,225.92943,225.929430,...,NaN,NaN,NaN,Inorganic,NaN,NaN,NaN,NaN,NaN,NaN
30,DTXSID2049715,Phosphonic acid,13598-36-2,ABLZXFCXXLZCGV-UHFFFAOYSA-N,Phosphonic acid,OP(O)=O,"InChI=1S/H3O3P/c1-4(2)3/h4H,(H2,1,2,3)",H3O3P,81.99500,81.981981,...,NaN,NaN,NaN,Inorganic,NaN,NaN,NaN,NaN,NaN,NaN
40,DTXSID0064830,"Silane, tetrafluoro-",7783-61-1,ABTOQLMXBSRXSM-UHFFFAOYSA-N,Tetrafluorosilane,F[Si](F)(F)F,"InChI=1S/F4Si/c1-5(2,3)4",F4Si,104.07900,103.970539,...,NaN,NaN,NaN,Inorganic,NaN,NaN,NaN,NaN,NaN,NaN
55,DTXSID4028042,Methyltin tris(2-ethylhexyl mercaptoacetate),57583-34-3,ACRNSUGLVQJCOM-UHFFFAOYSA-K,2-Ethylhexyl 10-ethyl-4-({2-[(2-ethylhexyl)oxy...,CCCCC(CC)COC(=O)CS[Sn](C)(SCC(=O)OCC(CC)CCCC)S...,InChI=1/3C10H20O2S.CH3.Sn/c3*1-3-5-6-9(4-2)7-1...,C31H60O6S3Sn,743.71000,744.257405,...,NaN,NaN,NaN,Organometallics,NaN,NaN,NaN,NaN,NaN,NaN
60,DTXSID90873902,Phosphinic acid,6303-21-5,ACVYVLVWPXVTIT-UHFFFAOYSA-N,Phosphinic acid,[H]P([H])(O)=O,"InChI=1S/H3O2P/c1-3-2/h3H2,(H,1,2)",H3O2P,65.99600,65.987066,...,NaN,NaN,NaN,Inorganic,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14175,DTXSID40883839,"Magnesium, chloroethyl-",2386-64-3,ZWNJJACKPNTXHY-UHFFFAOYSA-M,Chlorido(ethyl)magnesium,CC[Mg]Cl,"InChI=1S/C2H5.ClH.Mg/c1-2;;/h1H2,2H3;1H;/q;;+1...",C2H5ClMg,88.82000,87.993020,...,NaN,NaN,NaN,Organometallics,NaN,NaN,NaN,NaN,NaN,NaN
14189,DTXSID3064930,Titanium chloride (TiCl2),10049-06-6,ZWYDDDAMNQQZHD-UHFFFAOYSA-L,Titanium(2+) dichloride,Cl[Ti]Cl,InChI=1S/2ClH.Ti/h2*1H;/q;;+2/p-2,Cl2Ti,118.77000,117.885647,...,NaN,NaN,NaN,Inorganic,NaN,NaN,NaN,NaN,NaN,NaN
14193,DTXSID1061441,"Stannane, dibutyldimethoxy-",1067-55-6,ZXDVQYBUEVYUCG-UHFFFAOYSA-N,Dibutyl(dimethoxy)stannane,CCCC[Sn](CCCC)(OC)OC,InChI=1S/2C4H9.2CH3O.Sn/c2*1-3-4-2;2*1-2;/h2*1...,C10H24O2Sn,295.01000,296.079832,...,NaN,NaN,NaN,Organometallics,NaN,NaN,NaN,NaN,NaN,NaN
14212,DTXSID2037136,Uranium nitrate oxide (UO2(NO3)2),10102-06-4,ZYEWBKAZYICUMJ-UHFFFAOYSA-N,Bis(nitrato-kappaO)[bis(oxido)]uranium,[O-][N+](=O)O[U](=O)(=O)O[N+]([O-])=O,InChI=1S/2NO3.2O.U/c2*2-1(3)4;;;/q2*-1;;;+2,N2O8U,394.03500,394.016250,...,NaN,NaN,NaN,Inorganic,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
df3['physical_form'].value_counts(dropna = False)

solid     7690
liquid    5407
NaN       1029
gas        121
Name: physical_form, dtype: int64

In [51]:
#df3[['MP_pred', 'BP_pred', 'physical_form']]

Sanity check that the function is working as intended

In [52]:
df3[df3['BP_pred'] < 25][['MP_pred', 'BP_pred', 'physical_form']]

,MP_pred,BP_pred,physical_form
12,-121.0,-5.0,gas
118,-181.0,-81.0,gas
413,-187.0,-42.0,gas
466,157.0,-53.0,solid
500,-133.0,-1.0,gas
...,...,...,...
13223,-118.0,8.0,gas
13243,-168.0,20.0,gas
13321,-123.0,-19.0,gas
13623,-100.0,1.0,gas


In [53]:
df3

,dtxsid,PREFERRED_NAME,CASRN,INCHIKEY,IUPAC_NAME,SMILES,INCHI_STRING,MOLECULAR_FORMULA,AVERAGE_MASS,MONOISOTOPIC_MASS,...,AD_LogD,AD_index_LogD,Conf_index_LogD,errors,qsar_ready_smiles,HLC,WS_pred_mol/L,WS_pred_mg/L,test_track,physical_form
0,DTXSID3060164,Benzhydrylbenzene,519-73-3,AAAQKTZKLRYKHR-UHFFFAOYSA-N,"1,1',1''-Methanetriyltribenzene",C1=CC=C(C=C1)C(C1=CC=CC=C1)C1=CC=CC=C1,InChI=1S/C19H16/c1-4-10-16(11-5-1)19(17-12-6-2...,C19H16,244.337,244.125201,...,0.0,0.3140,0.3825,NaN,C1=CC=CC=C1C(C1C=CC=CC=1)C1C=CC=CC=1,2.630268e-06,4.073803e-07,0.099452,B,solid
1,DTXSID7060837,"Propane, 1,3-diiodo-",627-31-6,AAAXMNYUNVCMCJ-UHFFFAOYSA-N,"1,3-Diiodopropane",ICCCI,InChI=1S/C3H6I2/c4-2-1-3-5/h1-3H2,C3H6I2,295.890,295.855890,...,0.0,0.4990,0.4725,NaN,ICCCI,2.454709e-03,7.413102e-04,219.321006,B,liquid
2,DTXSID9025879,"3,3'-(p-Phenylene)diacrylic acid",16323-43-6,AAFXQFIGKBLKMC-UHFFFAOYSA-N,"3,3'-(1,4-Phenylene)di(prop-2-enoic acid)",OC(=O)C=CC1=CC=C(C=CC(O)=O)C=C1,InChI=1S/C12H10O4/c13-11(14)7-5-9-1-2-10(4-3-9...,C12H10O4,218.208,218.057909,...,1.0,0.5125,0.7140,NaN,OC(=O)C=CC1C=CC(C=CC(O)=O)=CC=1,6.025596e-10,9.120108e-03,1988.711764,B,solid
3,DTXSID2026282,C.I. Vat brown 3,131-92-0,AAKMSGQPNUGLAZ-UHFFFAOYSA-N,"N,N'-(5,10,15,17-Tetraoxo-10,15,16,17-tetrahyd...",O=C(NC1=CC=CC2=C1C(=O)C1=C(C3=C(C=C1)C1=C(N3)C...,InChI=1S/C42H23N3O6/c46-37-24-14-7-8-15-25(24)...,C42H23N3O6,665.661,665.158685,...,0.0,0.2025,0.4450,NaN,O=C(NC1=CC2=C(NC3C4=C(C=CC2=3)C(=O)C2C(=CC=CC=...,2.630268e-09,2.454709e-08,0.016377,B,solid
4,DTXSID4052188,"2,5,5-Trimethyl-2-phenyl-1,3-dioxane",5406-58-6,AALXTPRRKXUUOM-UHFFFAOYSA-N,"2,5,5-Trimethyl-2-phenyl-1,3-dioxane",CC1(C)COC(C)(OC1)C1=CC=CC=C1,"InChI=1S/C13H18O2/c1-12(2)9-14-13(3,15-10-12)1...",C13H18O2,206.285,206.130680,...,0.0,0.2825,0.3070,NaN,CC1(C)COC(C)(OC1)C1C=CC=CC=1,9.772372e-05,1.148154e-03,236.669687,B,solid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14242,DTXSID5064209,2-(Carboxymethylthio)benzothiazole,6295-57-4,ZZUQWNYNSKJLPI-UHFFFAOYSA-N,"[(1,3-Benzothiazol-2-yl)sulfanyl]acetic acid",OC(=O)CSC1=NC2=CC=CC=C2S1,InChI=1S/C9H7NO2S2/c11-8(12)5-13-9-10-6-3-1-2-...,C9H7NO2S2,225.280,224.991821,...,1.0,0.5305,0.7580,NaN,OC(=O)CSC1=NC2=CC=CC=C2S1,3.388442e-09,9.772372e-04,219.870381,B,solid
14243,DTXSID7062873,"Butanoic acid, 4-chloro-, methyl ester",3153-37-5,ZZUYIRISBMWFMV-UHFFFAOYSA-N,Methyl 4-chlorobutanoate,COC(=O)CCCCl,"InChI=1S/C5H9ClO2/c1-8-5(7)3-2-4-6/h2-4H2,1H3",C5H9ClO2,136.580,136.029107,...,0.0,0.3150,0.3670,NaN,COC(=O)CCCCl,2.818383e-04,2.041738e-01,27773.578977,B,liquid
14244,DTXSID9040342,"4-Ethyl-1,3-dioxolan-2-one",4437-85-8,ZZXUZKXVROWEIF-UHFFFAOYSA-N,"4-Ethyl-1,3-dioxolan-2-one",CCC1COC(=O)O1,"InChI=1/C5H8O3/c1-2-4-3-7-5(6)8-4/h4H,2-3H2,1H3",C5H8O3,116.116,116.047344,...,0.0,0.2740,0.2945,NaN,CCC1COC(=O)O1,2.570396e-08,9.772372e-01,113405.784063,B,liquid
14245,DTXSID3044889,"2,5-Di-tert-butylcyclohexa-2,5-diene-1,4-dione",2460-77-7,ZZYASVWWDLJXIM-UHFFFAOYSA-N,"2,5-Di-tert-butylcyclohexa-2,5-diene-1,4-dione",CC(C)(C)C1=CC(=O)C(=CC1=O)C(C)(C)C,"InChI=1S/C14H20O2/c1-13(2,3)9-7-12(16)10(8-11(...",C14H20O2,220.312,220.146330,...,0.0,0.3690,0.3790,NaN,CC(C)(C)C1=CC(=O)C(=CC1=O)C(C)(C)C,5.370318e-04,3.981072e-03,876.418325,B,solid


In [54]:
writer = pd.ExcelWriter(interim_dir+'TSCA_OPERA_predictions_080722.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.

df3.to_excel(writer, sheet_name = 'TSCA_OPERA')

writer.save()


In [57]:
df3.shape

(14247, 93)